In [1]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn

In [2]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)


C:\Users\zacfa\.julia\conda\3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print(len(trainset))

4446965


## Remove Illegal Match

In [4]:
trainset = trainset.drop(["matchType"],axis=1)

# Normalization and feature engineering

Add a feature for the number of players joined

In [5]:
trainset['playersJoined'] = trainset.groupby('matchId')['matchId'].transform('count')
trainset['killsNorm'] = trainset['kills']*((100-trainset['playersJoined'])/100 + 1)
trainset['damageDealtNorm'] = trainset['damageDealt']*((100-trainset['playersJoined'])/100 + 1)
trainset['maxPlaceNorm'] = trainset['maxPlace']*((100-trainset['playersJoined'])/100 + 1)
trainset['matchDurationNorm'] = trainset['matchDuration']*((100-trainset['playersJoined'])/100 + 1)

# Train on XGBoost

In [6]:
train_columns = list(trainset.columns[:-1])
x = trainset.drop(columns = ['winPlacePerc'])
#x = trainset
y = trainset["winPlacePerc"]


x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [7]:
x_train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,playersJoined,killsNorm,damageDealtNorm,maxPlaceNorm,matchDurationNorm
1648820,5762837079089987,51007019788459727,6752122462801886,0,0,0.00,0,0,0,40,...,0,0,232.3,2,0,96,0.00,0.0000,29.12,1966.64
690464,36434665515990655,41504757129991929,8296629515924295,1,2,106.90,0,0,5,47,...,0,0,2933.0,5,0,95,0.00,112.2450,50.40,2150.40
2371803,26008611844141230,21780241820167115,26757196943090263,0,0,0.00,0,0,0,73,...,0,0,68.7,1,0,85,0.00,0.0000,29.90,1654.85
3541716,5052800135186152,60941483575078955,4483006274301557,0,4,64.78,0,0,0,43,...,0,0,1511.0,4,1514,96,0.00,67.3712,49.92,1807.52
2291,12770305061069884,16337098749221580,53754644994239682,1,4,214.90,1,0,8,25,...,0,0,2003.0,2,0,96,1.04,223.4960,28.08,1363.44


In [8]:
y_train.head()

1648820    0.8148
690464     0.7872
2371803    0.0400
3541716    0.7872
2291       0.6538
Name: winPlacePerc, dtype: float64

In [9]:
trainset_dmatrix = xgb.DMatrix(x_train.values,label=y_train.values)
valset_dmatrix = xgb.DMatrix(x_val.values,label=y_val.values)

In [12]:
params = {
    "max_depth" : 10,
    "eval_metric" : ["mae"],
}

In [13]:
clf = xgb.train(params, trainset_dmatrix, evals=[(trainset_dmatrix, "train"),(valset_dmatrix, 'val')], num_boost_round = 10)

predictions = xgb.predict(valset_dmatrix)

[14:05:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1914 extra nodes, 0 pruned nodes, max_depth=10
[0]	train-mae:0.196089	val-mae:0.196003
[14:05:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1914 extra nodes, 0 pruned nodes, max_depth=10
[1]	train-mae:0.146703	val-mae:0.146681
[14:06:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1944 extra nodes, 0 pruned nodes, max_depth=10
[2]	train-mae:0.113851	val-mae:0.113853
[14:06:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1960 extra nodes, 0 pruned nodes, max_depth=10
[3]	train-mae:0.092791	val-mae:0.092843
[14:06:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1964 extra nodes, 0 pruned nodes, max_depth=10
[4]	train-mae:0.079195	val-mae:0.079297
[14:07:00] C:\Users\Administra

AttributeError: module 'xgboost' has no attribute 'predict'